## Eksperimen 1: Visual Problem Generation (VLLM)

Generate soal visual dengan VLLM.

In [161]:
print("Setup VLLM experiment...")

Setup VLLM experiment...


In [162]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [163]:
%pip install -q openai requests

from openai import OpenAI
import time, json, re, datetime, requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [164]:
resp = requests.get("http://localhost:8000/v1/models").json()
resp


{'object': 'list',
 'data': [{'id': 'Qwen/Qwen1.5-0.5B-Chat',
   'object': 'model',
   'created': 1769332595,
   'owned_by': 'vllm',
   'root': 'Qwen/Qwen1.5-0.5B-Chat',
   'parent': None,
   'max_model_len': 1024,
   'permission': [{'id': 'modelperm-996594e4a29174aa',
     'object': 'model_permission',
     'created': 1769332595,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': True,
     'allow_search_indices': False,
     'allow_view': True,
     'allow_fine_tuning': False,
     'organization': '*',
     'group': None,
     'is_blocking': False}]}]}

In [165]:
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY"
)

In [166]:
def build_prompt(plan, date_str):
    return f"""
TUGAS: Buat SATU soal matematika BARU sesuai rencana berikut:
{plan}

Soal HARUS tentang menghitung LUAS bangun datar gabungan PERSEGI dan SEGITIGA.

Berikut CONTOH OUTPUT YANG BENAR (JANGAN DISALIN, hanya sebagai pola):

{{
  "soal": "Sebuah persegi memiliki sisi 8 cm. Pada salah satu sisinya ditempel segitiga siku-siku dengan alas 8 cm dan tinggi 6 cm. Hitung luas total bangun datar gabungan tersebut.",
  "solution": "Luas persegi = 8 × 8 = 64 cm². Luas segitiga = 1/2 × 8 × 6 = 24 cm². Jadi luas total = 88 cm²."
}}

Sekarang buat soal BARU dengan angka berbeda.

ATURAN:
- HARUS ada angka.
- HARUS ada perhitungan luas.
- HARUS berbentuk soal cerita.
- Output HANYA JSON.
- JSON hanya punya: soal, solution.
"""


In [167]:
def run_experiment_vllm(plan, date_str):
    prompt = build_prompt(plan, date_str)

    start = time.time()

    response = client.chat.completions.create(
        model="Qwen/Qwen1.5-0.5B-Chat",
        messages=[
            {"role": "system", "content": "You generate concrete math problems with numbers and calculations. You only output valid JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=400,
        response_format={"type": "json_object"}
    )

    end = time.time()
    return response.choices[0].message.content, end - start


In [168]:
plan = "Geometri: Menghitung luas bangun datar gabungan (Persegi dan Segitiga)"
date = datetime.date.today().strftime("%Y-%m-%d")

raw, t = run_experiment_vllm(plan, date)

print("Waktu inferensi:", t)
try:
    data = json.loads(raw)
    print("\nPARSED JSON:\n", json.dumps(data, indent=4, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("\n❌ JSON rusak, raw output disimpan:")
    print(raw)



Waktu inferensi: 1.0912373065948486

PARSED JSON:
 {
    "soal": "Sebuah persegi memiliki sisi 8 cm. Pada salah satu sisinya ditempel segitiga siku-siku dengan alas 8 cm dan tinggi 6 cm. Hitung luas total bangun datar gabungan tersebut.",
    "solution": "Luas persegi = 8 × 8 = 64 cm². Luas segitiga = 1/2 × 8 × 6 = 24 cm². Jadi luas total = 88 cm²."
}
